In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

In [2]:
park_info = pd.read_csv('../data/park_info.csv')

In [3]:
park_code_ = park_info['parkCode'].unique()

# Convert values to uppercase and store them in a list
park_code_list = [value.upper() for value in park_code_]

In [4]:
len(park_code_list)

462

In [5]:
park_code_list

['BICR',
 'FRRI',
 'HOBE',
 'LIRI',
 'NATT',
 'NATR',
 'RUCA',
 'SEMO',
 'TRTE',
 'TUAI',
 'TUIN',
 'ALAG',
 'ANCH',
 'ALEU',
 'ANIA',
 'BELA',
 'CAKR',
 'DENA',
 'GAAR',
 'GLBA',
 'INUP',
 'KATM',
 'KEFJ',
 'KLGO',
 'KOVA',
 'LACL',
 'NOAT',
 'SITK',
 'WRST',
 'YUCH',
 'BUOV',
 'CACH',
 'CAGR',
 'CHIR',
 'CORO',
 'FOBO',
 'GLCA',
 'GRCA',
 'PARA',
 'HUTR',
 'JUBA',
 'LAKE',
 'MOCA',
 'NAVA',
 'OLSP',
 'ORPI',
 'PEFO',
 'PISP',
 'SAGU',
 'SUCR',
 'TONT',
 'TUMA',
 'TUZI',
 'WACA',
 'WUPA',
 'ARPO',
 'BUFF',
 'FOSM',
 'HOSP',
 'CHSC',
 'PERI',
 'WICL',
 'ALCA',
 'CABR',
 'CALI',
 'CAMO',
 'CHIS',
 'CECH',
 'DEVA',
 'DEPO',
 'EUON',
 'FOPO',
 'GOGA',
 'JOMU',
 'JOTR',
 'LAVO',
 'LABE',
 'MANZ',
 'MOJA',
 'MUWO',
 'PINN',
 'PORE',
 'POEX',
 'POCH',
 'PRSF',
 'REDW',
 'RORI',
 'SAFR',
 'SAMO',
 'SEKI',
 'TULE',
 'WHIS',
 'YOSE',
 'AMCH',
 'BEOL',
 'BLCA',
 'COLM',
 'CURE',
 'DINO',
 'FLFO',
 'GRSA',
 'HOVE',
 'MEVE',
 'ROMO',
 'SAND',
 'SAFE',
 'YUHO',
 'APPA',
 'COLT',
 'NEEN',
 'WARO',
 

In [ ]:
park_code = "HOBE"
url = f"https://irma.nps.gov/Stats/MvcReportViewer.aspx?_id=7e661565-c923-497d-8d4d-8fc0dcc43632&_m=Remote&_r=/NPS.Stats.Reports/Park Specific Reports/Recreation Visitors By Month (1979 - Last Calendar Year)&_15=True&_16=True&_18=True&_19=True&_34=False&_35=False&_39=880px&Park={park_code}"
page = requests.get(url)
if page.status_code == 200:
    soup = BeautifulSoup(page.content, "html.parser")
    #print(soup.prettify())
    target_div = soup.find("div")
    tables = target_div.find_all('table')

    result=[]
    for table in tables:
        # Extract data from each table
        rows = table.find_all('td')
        for column in rows:
            result.append(column)
    visitor_data=[]
    for item in result:
        visitor_data.append(item.text)
    visitor_data_c = visitor_data[105:749]
    num_columns = 14
    num_rows = len(visitor_data_c) // num_columns
    reshaped_data = pd.DataFrame(visitor_data_c).values.reshape(num_rows, num_columns)
    #visitor_df = pd.DataFrame(reshaped_data[1:], columns=reshaped_data[0])
else:
    print(f"The URL returned a response. Moving on to the next URL.")

In [ ]:
pd.DataFrame(reshaped_data[1:], columns=reshaped_data[0])

In [31]:
visitor_dfs=[]
for codes in park_code_list:
    url = f"https://irma.nps.gov/Stats/MvcReportViewer.aspx?_id=7e661565-c923-497d-8d4d-8fc0dcc43632&_m=Remote&_r=/NPS.Stats.Reports/Park Specific Reports/Recreation Visitors By Month (1979 - Last Calendar Year)&_15=True&_16=True&_18=True&_19=True&_34=False&_35=False&_39=880px&Park={codes}"
    page = requests.get(url)
    if page.status_code == 200:
        try:
            soup = BeautifulSoup(page.content, "html.parser")
            target_div = soup.find("div")
            tables = target_div.find_all('table')

            result=[]
            for table in tables:
            # Extract data from each table
                rows = table.find_all('td')
                for column in rows:
                    result.append(column)
            
            visitor_data=[]
            for item in result:
                visitor_data.append(item.text)
            visitor_data = visitor_data[105:749]
            num_columns = 14
            num_rows = len(visitor_data) // num_columns
            visitor_df = pd.DataFrame(visitor_data).values.reshape(num_rows, num_columns)
            visitor_df_2 = pd.DataFrame(visitor_df)
            visitor_df_2['parkCode'] = codes
            visitor_dfs.append(visitor_df_2)
        except Exception as e:
            continue


In [32]:
len(visitor_dfs)

451

In [36]:
park_visitor_data = pd.concat(visitor_dfs[2:], ignore_index=True)

In [39]:
park_visitor_data.to_csv('../data/park_visitor_data.csv')

In [40]:
park_visitor_data.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,parkCode
0,Year,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,Total,HOBE
1,2023,"3,833","4,575","8,030","6,448","8,759","7,702","7,067","7,361","6,101","5,468","4,269","3,112","72,725",HOBE
2,2022,"4,278","3,791","6,518","5,934","7,408","7,501","6,628","6,294","5,864","5,565","4,756","3,236","67,773",HOBE
3,2021,"1,555","2,146","3,907","3,382","6,587","6,967","7,505","6,532","6,252","5,958","4,567","2,927","58,285",HOBE
4,2020,"1,373","1,754","3,382",747,910,"4,425","4,713","3,722","2,586","2,042","1,686","1,260","28,600",HOBE
5,2019,96,"1,961","6,644","3,756","6,528","5,381","8,193","3,853","2,387","2,348","2,122","2,103","45,372",HOBE
6,2018,"2,265","3,495","8,948","4,731","5,394","5,248","5,992","3,796","3,424","2,825","2,116",257,"48,491",HOBE
7,2017,"4,548","5,224","9,155","9,218","8,237","6,537","7,715","5,290","5,259","4,657","5,429","2,986","74,255",HOBE
8,2016,"3,476","5,050","9,691","8,309","8,214","8,438","8,105","7,992","5,913","8,533","5,065","4,584","83,370",HOBE
9,2015,"3,942","4,192","10,685","6,995","10,354","8,378","7,992","7,088","6,098","6,517","5,576","11,465","89,282",HOBE


In [41]:
park_visitor_data.tail(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,parkCode
16826,1988,"9,943","10,894","13,991","20,433","89,014","268,452","320,432","292,705","125,016","54,972","17,453","9,386","1,232,691",GRTE
16827,1987,"11,612","11,088","12,310","21,241","74,134","230,020","398,516","381,568","220,715","60,499","19,071","10,017","1,450,791",GRTE
16828,1986,"9,625","9,028","14,040","11,553","64,026","229,917","374,139","361,709","162,631","46,002","13,994","9,658","1,306,322",GRTE
16829,1985,"8,656","8,868","10,414","13,325","51,266","238,661","386,954","368,768","186,167","40,346","10,379","10,679","1,334,483",GRTE
16830,1984,"8,226","9,788","10,099","11,344","53,385","228,142","374,365","380,301","215,255","48,538","13,681","7,774","1,360,898",GRTE
16831,1983,"9,966","10,935","12,066","11,526","119,010","231,024","370,515","444,690","233,269","56,456","26,324","6,254","1,532,035",GRTE
16832,1982,"15,098","26,809","20,773","19,308","146,341","481,909","790,452","720,615","138,944","101,968","52,787","19,025","2,534,029",GRTE
16833,1981,"18,959","31,878","25,886","26,846","176,726","451,018","674,566","699,874","357,057","106,157","52,768","21,909","2,643,644",GRTE
16834,1980,"15,804","31,020","34,943","29,195","136,336","395,897","635,656","777,149","309,929","95,773","43,950","49,975","2,555,627",GRTE
16835,1979,"46,131","24,585","32,175","38,696","134,499","384,945","608,613","667,299","305,194","123,444","46,685","33,905","2,446,171",GRTE
